# CTMRG Tutorial: From 1D Transfer Matrix to 2D Corner Transfer Matrix

## A Comprehensive Guide to Corner Transfer Matrix Renormalization Group

---

**Prerequisites:**
- Understanding of Matrix Product States (MPS)
- Basic knowledge of infinite DMRG (iDMRG)
- Familiarity with 1D Ising model transfer matrix

**Learning Objectives:**
1. Understand how 1D transfer matrix generalizes to 2D
2. Grasp the concept of "Corner" in transfer matrix context
3. Learn the RG procedure for truncating the corner matrices
4. See how CTMRG connects to tensor network methods

---

## Table of Contents

1. [Review: 1D Transfer Matrix](#1-review-1d-transfer-matrix)
2. [From 1D to 2D: The Challenge](#2-from-1d-to-2d-the-challenge)
3. [The 2D Transfer Matrix](#3-the-2d-transfer-matrix)
4. [What is a Corner Transfer Matrix?](#4-what-is-a-corner-transfer-matrix)
5. [Why Do We Need Corners?](#5-why-do-we-need-corners)
6. [The Renormalization Group Idea](#6-the-renormalization-group-idea)
7. [CTMRG Algorithm Overview](#7-ctmrg-algorithm-overview)
8. [Connection to Tensor Networks](#8-connection-to-tensor-networks)
9. [Summary and Next Steps](#9-summary-and-next-steps)

---

# 1. Review: 1D Transfer Matrix

Let's start from what you already know - the 1D Ising model transfer matrix.

## 1.1 The 1D Ising Model

The 1D Ising model has the Hamiltonian:

$$H = -J \sum_{i} \sigma_i \sigma_{i+1} - h \sum_i \sigma_i$$

where $\sigma_i \in \{+1, -1\}$.

The partition function is:

$$Z = \sum_{\{\sigma\}} e^{-\beta H} = \sum_{\sigma_1, \sigma_2, \ldots, \sigma_N} \prod_i e^{\beta J \sigma_i \sigma_{i+1} + \beta h \sigma_i}$$

## 1.2 The Transfer Matrix Trick

The key insight is that we can write the partition function as a **matrix product**:

$$Z = \sum_{\sigma_1, \ldots, \sigma_N} T_{\sigma_1 \sigma_2} T_{\sigma_2 \sigma_3} \cdots T_{\sigma_{N-1} \sigma_N} T_{\sigma_N \sigma_1}$$

where the transfer matrix $T$ is a $2 \times 2$ matrix:

$$T = \begin{pmatrix} e^{\beta J + \beta h} & e^{-\beta J} \\ e^{-\beta J} & e^{\beta J - \beta h} \end{pmatrix}$$

This can be written compactly as:

$$Z = \text{Tr}(T^N)$$

## 1.3 Why is Transfer Matrix Useful?

```
Physical picture: "Transfer" from site to site

    σ₁ ──T── σ₂ ──T── σ₃ ──T── σ₄ ──T── σ₅ ──T── ...
    
Each T "transfers" the Boltzmann weight from one site to the next.
```

**Key advantages:**
1. In thermodynamic limit ($N \to \infty$): $Z \approx \lambda_{\max}^N$ (largest eigenvalue dominates)
2. Free energy: $f = -k_B T \ln \lambda_{\max}$
3. Correlations: Can be computed from eigenvectors

**This is a CRUCIAL concept:** The transfer matrix encodes the local Boltzmann weights, and by "transferring" through the system, we can compute global properties.

---

# 2. From 1D to 2D: The Challenge

Now let's see what happens when we go to 2D.

## 2.1 The 2D Ising Model

On a square lattice:

$$H = -J \sum_{\langle i,j \rangle} \sigma_i \sigma_j$$

where $\langle i,j \rangle$ means nearest neighbors (both horizontal and vertical).

```
    σ₁₁ ── σ₁₂ ── σ₁₃ ── σ₁₄
     |      |      |      |
    σ₂₁ ── σ₂₂ ── σ₂₃ ── σ₂₄
     |      |      |      |
    σ₃₁ ── σ₃₂ ── σ₃₃ ── σ₃₄
     |      |      |      |
    σ₄₁ ── σ₄₂ ── σ₄₃ ── σ₄₄
```

## 2.2 Can We Still Use Transfer Matrix?

**Yes!** But now the transfer matrix "transfers" an **entire row** (or column).

```
Row 1:  σ₁₁ ── σ₁₂ ── σ₁₃ ── σ₁₄   ← This is like ONE "super-spin"
              ↓ T (transfer)
Row 2:  σ₂₁ ── σ₂₂ ── σ₂₃ ── σ₂₄   ← Another "super-spin"
              ↓ T (transfer)
Row 3:  σ₃₁ ── σ₃₂ ── σ₃₃ ── σ₃₄
              ↓ T (transfer)
        ...
```

## 2.3 The Problem: Exponential Growth!

If the row has $L$ spins, each spin can be $\pm 1$, so:
- A "row configuration" has $2^L$ possibilities
- The transfer matrix is $2^L \times 2^L$

For $L = 10$: matrix is $1024 \times 1024$ — still OK

For $L = 20$: matrix is $\sim 10^6 \times 10^6$ — already hard

For $L = 100$: matrix is $2^{100} \times 2^{100}$ — **IMPOSSIBLE!**

**This is the fundamental challenge:** In 2D, the transfer matrix grows exponentially with system size.

💡 **Key insight:** We need a way to **approximate** this exponentially large transfer matrix. This is where CTMRG comes in!

---

# 3. The 2D Transfer Matrix

Let's be more precise about the 2D row-to-row transfer matrix.

## 3.1 Row-to-Row Transfer Matrix

Consider two adjacent rows with configurations $\{\sigma\}$ and $\{\sigma'\}$:

```
Row n:    σ₁ ── σ₂ ── σ₃ ── σ₄ ── ... ── σ_L
           |     |     |     |            |
Row n+1:  σ'₁ ── σ'₂ ── σ'₃ ── σ'₄ ── ... ── σ'_L
```

The transfer matrix element is:

$$T_{\{\sigma\}, \{\sigma'\}} = \prod_{i=1}^{L} e^{\beta J \sigma_i \sigma'_i} \cdot \prod_{i=1}^{L-1} e^{\beta J \sigma'_i \sigma'_{i+1}}$$

- First product: vertical bonds between rows
- Second product: horizontal bonds within the new row

## 3.2 Partition Function in 2D

For an $L \times M$ lattice with periodic boundary conditions:

$$Z = \text{Tr}(T^M)$$

Same formula as 1D! But $T$ is now a $2^L \times 2^L$ matrix.

## 3.3 Visual: The Transfer Matrix as a "Slab"

Think of the transfer matrix as a **slab** that adds one row at a time:

```
                    ┌─────────────────┐
    Configuration   │                 │   Configuration
    of row n    ────│   Transfer T    │──── of row n+1
    (2^L states)    │                 │    (2^L states)
                    └─────────────────┘
                    
    This slab contains all Boltzmann weights for:
    - Vertical bonds (connecting the two rows)
    - Horizontal bonds (within the bottom row)
```

---

# 4. What is a Corner Transfer Matrix?

Now we arrive at the **"Corner"** concept. This is the key innovation!

## 4.1 Baxter's Insight (1968-1982)

Rodney Baxter, while solving the 2D Ising and other exactly solvable models, realized:

> Instead of transferring row-by-row, what if we build the lattice **corner-by-corner**?

## 4.2 Dividing the Lattice into Quadrants

Imagine an infinite lattice divided into 4 quadrants by a "+" shape:

```
         ↑
    C₄   │   C₁
         │
  ───────┼───────→
         │
    C₃   │   C₂
         │
         
C₁ = Upper-right corner
C₂ = Lower-right corner
C₃ = Lower-left corner
C₄ = Upper-left corner
```

Each **corner** $C_i$ represents the partition function contribution from that quadrant!

## 4.3 What Does a Corner Matrix Look Like?

Focus on the upper-right corner $C_1$:

```
                     │
    · · · · σ_a1 ─── σ_b1
                │     │
    · · · · σ_a2 ─── σ_b2
                │     │
    · · · · σ_a3 ─── σ_b3
    ────────────┼─────────
                │
          (central site)
                
    {σ_a} = spins along vertical edge (left boundary of C₁)
    {σ_b} = spins along horizontal edge (bottom boundary of C₁)
```

The **Corner Transfer Matrix** $C_1$ has matrix elements:

$$(C_1)_{\{\sigma_a\}, \{\sigma_b\}} = \sum_{\text{spins inside corner}} (\text{Boltzmann weights in the corner})$$

It connects:
- Row indices: configuration $\{\sigma_a\}$ along the vertical edge
- Column indices: configuration $\{\sigma_b\}$ along the horizontal edge

## 4.4 Intuition: Corner as a "Quarter Pie"

```
    If the full lattice is a pie:
    
         ╱───────╲
        ╱    │    ╲
       │  C₄ │ C₁  │
       │─────┼─────│
       │  C₃ │ C₂  │
        ╲    │    ╱
         ╲───────╱
         
    Each Cᵢ is one quarter of the pie!
```

The full partition function (for a large enough system) can be written as:

$$Z \approx \text{Tr}(C_1 C_2 C_3 C_4)$$

This is **beautiful**: we're building the 2D partition function by connecting four corners!

---

# 5. Why Do We Need Corners?

You might ask: "Why not just use the row transfer matrix?"

## 5.1 The Advantage of Corners

**Key insight:** Corners are better suited for:
1. Computing **local properties** (like magnetization at the center)
2. Implementing **renormalization** in a natural way
3. Generalizing to **infinite** 2D systems

## 5.2 Computing Local Observables

To compute $\langle \sigma_{\text{center}} \rangle$:

```
         C₄   │   C₁
              │
       ───────●───────  ← measure σ at the center
              │
         C₃   │   C₂
```

$$\langle \sigma_{\text{center}} \rangle = \frac{\text{Tr}(C_1 C_2 C_3 C_4 \cdot \sigma_{\text{center}})}{\text{Tr}(C_1 C_2 C_3 C_4)}$$

The corners naturally "surround" the point where we measure!

## 5.3 Another Key Object: Half-Row/Half-Column Transfer Matrices

Besides corners, we also need **edge transfer matrices** (often called $T_1, T_2, T_3, T_4$):

```
                T₄ (top edge)
              ─────────────
         C₄   │           │   C₁
              │           │
    T₃ (left) │     ●     │ T₁ (right edge)
              │           │
         C₃   │           │   C₂
              ─────────────
                T₂ (bottom edge)
```

These $T_i$ matrices transfer along the edges (half-infinite rows or columns).

## 5.4 The Full Environment

The **environment** of a central site consists of:
- 4 corners: $C_1, C_2, C_3, C_4$
- 4 edge transfers: $T_1, T_2, T_3, T_4$

Together, they encode the effect of the **entire infinite lattice** on local observables.

---

# 6. The Renormalization Group Idea

Now we understand what corners are. But they're still exponentially large! This is where **RG** comes in.

## 6.1 The Problem Revisited

If the corner has size $L \times L$:
- The boundary has $L$ spins
- Each boundary has $2^L$ configurations
- Corner matrix is $2^L \times 2^L$

For infinite corners, this is hopeless... unless we **truncate**!

## 6.2 The RG Philosophy

The RG idea (familiar from DMRG):

> Not all $2^L$ configurations are equally important. Keep only the most **relevant** ones!

In DMRG, you learned that:
- The "relevant" states are determined by the **environment**
- We keep states with largest **eigenvalues** of the density matrix
- This corresponds to keeping states with largest **entanglement** contribution

## 6.3 RG for Corners

The same philosophy applies to corners!

**Step 1:** Start with a small corner
```
    Small corner (e.g., 2×2 region)
    
    σ_a ─── σ
      │     │
    σ_b ───[·]
```

**Step 2:** "Grow" the corner by adding sites
```
    Add one row/column at a time
    
    σ'_a ─── σ_a ─── σ
      │       │      │
    σ'_b ─── σ_b ───[·]
      │       │
     [·] ───[·]
```

**Step 3:** **Truncate** to keep only $\chi$ states (bond dimension)

This is the **renormalization** step! We're coarse-graining the corner.

## 6.4 How to Truncate? (The Key Question)

In DMRG, truncation is based on the density matrix eigenvalues.

In CTMRG, truncation is based on **how the corners connect**:

```
    The "environment" for deciding which states to keep:
    
         C₄ ─── T₄ ─── C₁
          │             │
         T₃      ●     T₁
          │             │
         C₃ ─── T₂ ─── C₂
          
    Use the OTHER corners/edges to determine
    which states of THIS corner to keep.
```

This is analogous to DMRG's "system-environment" picture!

---

# 7. CTMRG Algorithm Overview

Now let's put it all together into the actual algorithm.

## 7.1 The CTMRG Setup

We work with:
- **4 corner matrices:** $C_1, C_2, C_3, C_4$ (each of size $\chi \times \chi$)
- **4 edge tensors:** $T_1, T_2, T_3, T_4$ (each of size $\chi \times d \times \chi$)
- **Local tensor:** $a$ (the Boltzmann weight tensor for one site)

Here $\chi$ is the **bond dimension** (truncation parameter) and $d$ is the physical dimension.

## 7.2 The Local Tensor for 2D Ising

For the 2D Ising model, the local tensor $a$ has 4 indices (up, down, left, right):

```
              up
               │
    left ──── [a] ──── right
               │
             down
```

$$a_{u,d,l,r} = \text{(Boltzmann weight when neighbors are } u, d, l, r \text{)}$$

For Ising: $a_{u,d,l,r} = \sum_\sigma W(\sigma, u) W(\sigma, d) W(\sigma, l) W(\sigma, r)$

where $W(\sigma, \sigma') = e^{\beta J \sigma \sigma' / 2}$ (split the bond weight symmetrically).

## 7.3 The CTMRG Iteration

The algorithm iterates until convergence:

### Step 1: Absorb - Grow the corners

```
    Absorb the local tensor into the corner:
    
    Before:     After absorbing:
    
    C₁ ─── T₁       C₁' (larger)
     │      │    →   │
    T₄ ─── a        T₁'
    
    C₁' = C₁ · T₁ · T₄ · a (schematically)
```

The corner grows by incorporating one more local tensor.

### Step 2: Renormalize - Truncate back to χ

After absorption, the corner is too large ($\chi \cdot d$ instead of $\chi$).

We need to find **projectors** $P$ that truncate back:

```
    C₁' (size χd × χd)  →  C₁_new (size χ × χ)
    
    C₁_new = P† · C₁' · P
```

The projector $P$ is computed from the **environment** — this is the RG step!

### Step 3: Repeat for all four corners

Apply the absorption-renormalization to all four corners (by symmetry or one-by-one).

### Step 4: Check convergence

Iterate until the corner matrices (or observables) converge.

## 7.4 Computing the Projector

The projector is computed from the "environment" density matrix:

$$\rho = C_1' \cdot C_2' \cdot C_3' \cdot C_4' \quad \text{(contracted appropriately)}$$

Then:
1. Diagonalize $\rho$ (or do SVD on a related matrix)
2. Keep the $\chi$ largest eigenvalues/singular values
3. The corresponding eigenvectors form the projector $P$

This is **exactly** the DMRG philosophy: keep the most entangled/relevant states!

---

# 8. Connection to Tensor Networks

Since you know MPS, let's connect CTMRG to the tensor network language.

## 8.1 2D Tensor Networks

A 2D tensor network (like PEPS) represents a 2D wavefunction:

```
    A ─── A ─── A ─── A
    │     │     │     │
    A ─── A ─── A ─── A
    │     │     │     │
    A ─── A ─── A ─── A
    │     │     │     │
    A ─── A ─── A ─── A
    
    Each A is a local tensor with 5 indices:
    - 4 virtual indices (connecting to neighbors)
    - 1 physical index (the local spin)
```

## 8.2 The Contraction Problem

To compute expectation values, we need to **contract** the entire network:

$$\langle O \rangle = \frac{\langle \psi | O | \psi \rangle}{\langle \psi | \psi \rangle}$$

The denominator $\langle \psi | \psi \rangle$ requires contracting:

```
    a ─── a ─── a ─── a       where a = A† · A
    │     │     │     │       (physical index contracted)
    a ─── a ─── a ─── a
    │     │     │     │
    a ─── a ─── a ─── a
    │     │     │     │
    a ─── a ─── a ─── a
```

This 2D contraction is **exponentially hard** exactly (same problem as before!).

## 8.3 CTMRG as a Contraction Method

CTMRG provides an **approximate** way to contract 2D tensor networks!

```
    The environment tensors (C, T) approximate the infinite sum:
    
           C₄ ── T₄ ── T₄ ── T₄ ── C₁
            │                       │
           T₃    a ─── a ─── a     T₁
            │    │     │     │      │
           T₃    a ─── a ─── a     T₁    ≈    C₄ ── T₄ ── C₁
            │    │     │     │      │          │           │
           T₃    a ─── a ─── a     T₁         T₃    ●     T₁
            │                       │          │           │
           C₃ ── T₂ ── T₂ ── T₂ ── C₂        C₃ ── T₂ ── C₂
           
    CTMRG compresses the infinite environment into
    a finite set of tensors!
```

## 8.4 Modern CTMRG for iPEPS

Today, CTMRG is most commonly used to:
1. Contract infinite PEPS (iPEPS) wavefunctions
2. Compute expectation values in 2D quantum systems
3. Study 2D classical stat mech models (like Ising)

The connection to your DMRG knowledge:

| Aspect | DMRG (1D) | CTMRG (2D) |
|--------|-----------|------------|
| Target | Ground state of 1D chain | Contraction of 2D network |
| Representation | MPS | Corner + Edge tensors |
| Truncation | Density matrix eigenvalues | Environment-based projectors |
| Bond dimension | $\chi$ (MPS bond dim) | $\chi$ (environment bond dim) |
| Iteration | Sweep left-right | Absorb-renormalize corners |

---

# 9. Summary and Next Steps

## 9.1 What We Learned

1. **Transfer Matrix (Review):**
   - In 1D, transfers Boltzmann weight from site to site
   - In 2D, transfers row to row (exponentially large)

2. **Corner Transfer Matrix:**
   - Divides 2D lattice into 4 quadrants
   - Each corner encodes partition function of a quadrant
   - Natural for computing local observables

3. **Renormalization Group:**
   - Corners grow exponentially → need truncation
   - Keep most "relevant" states (largest eigenvalues)
   - Same philosophy as DMRG!

4. **CTMRG Algorithm:**
   - Iteratively: Absorb (grow) → Renormalize (truncate)
   - Converges to fixed-point environment tensors
   - Accuracy controlled by bond dimension $\chi$

5. **Connection to Tensor Networks:**
   - CTMRG contracts 2D tensor networks approximately
   - Essential tool for iPEPS and 2D physics

## 9.2 Conceptual Summary Diagram

```
    Your Journey:
    
    1D Ising Transfer Matrix
           │
           ▼
    2D Row-to-Row Transfer (exponentially large!)
           │
           ▼
    Corner Transfer Matrix (divide into quadrants)
           │
           ▼
    CTMRG: Absorb + Renormalize (truncate to χ)
           │
           ▼
    Efficient 2D Tensor Network Contraction!
```

## 9.3 Next Steps for Learning

1. **Implement 2D Ising CTMRG:**
   - Start with exact small corners
   - Add absorption step
   - Add renormalization (SVD truncation)
   - Compare with exact results at critical point

2. **Study the literature:**
   - Nishino & Okunishi (1996): Modern CTMRG formulation
   - Orús & Vidal (2009): CTMRG for iPEPS
   - Corboz et al.: Many applications to quantum systems

3. **Connect to your DMRG knowledge:**
   - The truncation step is conceptually identical
   - The "environment" picture transfers directly

## 9.4 Key Takeaway

> **CTMRG = Corner decomposition + Transfer matrix growth + RG truncation**
>
> It's a way to efficiently represent and contract infinite 2D systems,
> just like DMRG efficiently represents infinite 1D systems!

---

# Appendix A: Quick Reference Card

## Objects in CTMRG

| Symbol | Name | Size | Role |
|--------|------|------|------|
| $C_1, C_2, C_3, C_4$ | Corner matrices | $\chi \times \chi$ | Encode quadrant partition functions |
| $T_1, T_2, T_3, T_4$ | Edge tensors | $\chi \times d \times \chi$ | Transfer along half-infinite edges |
| $a$ | Local tensor | $d \times d \times d \times d$ | Boltzmann weights at one site |
| $P$ | Projector/Isometry | $\chi d \times \chi$ | Truncation matrix from RG |
| $\chi$ | Bond dimension | scalar | Controls accuracy (like $\chi$ in MPS) |

## Algorithm Summary

```
Initialize: C₁, C₂, C₃, C₄, T₁, T₂, T₃, T₄

Repeat until converged:
    1. Absorption:
       - Grow corners by absorbing local tensors and edges
       - C_i → C_i' (size χd × χd)
       - T_i → T_i' (size χd × d × χd)
       
    2. Renormalization:
       - Compute environment density matrix
       - SVD/eigendecomposition to get projector P
       - Truncate: C_i_new = P† C_i' P (size χ × χ)
       
    3. Check convergence:
       - Compare C_i_new with C_i_old
       - Or check convergence of observables
       
Output: Converged environment tensors
```

---

# Appendix B: Glossary

- **Transfer Matrix:** A matrix that encodes the Boltzmann weight for transitioning from one configuration to another (row-to-row in 2D).

- **Corner Transfer Matrix (CTM):** A matrix encoding the partition function contribution from one quadrant of the lattice.

- **Renormalization Group (RG):** A systematic way to "coarse-grain" a system by keeping only the most relevant degrees of freedom.

- **Bond Dimension ($\chi$):** The number of states kept after truncation; controls accuracy vs computational cost.

- **Projector/Isometry:** A matrix that maps from a larger space to a smaller truncated space, preserving the most important states.

- **Environment:** In DMRG/CTMRG, the "rest of the system" that determines which states to keep.

- **Absorption:** The step where we "grow" the corner by adding more sites/tensors.

- **iPEPS:** Infinite Projected Entangled Pair States, a 2D tensor network for quantum many-body states.

- **Contraction:** The process of summing over shared indices in a tensor network to compute a scalar (like partition function or expectation value).

---

*This tutorial was created as part of the CompPhysCTMRG project.*

*Happy learning! 🎓*